In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Import**

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# Importing dataset from Kaggle

In [ ]:
train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

In [ ]:
test.shape

## Pre Processing Test CSV

In [ ]:
test_X = []
# test_Y = []

for i in test.values:
    test_X.append(i.reshape(28,28))
       
test_X = np.array(test_X)
# test_Y = np.array(test_Y)

In [ ]:
test_X[0].shape

## Scaling Test csv

In [ ]:
test_X = test_X / 256


In [ ]:
test_X[0]

## Reshaping test csv

In [ ]:
test_X = test_X.reshape(-1,28,28,1)

In [ ]:
test_X.shape

In [ ]:
train.shape

In [ ]:
train.head()

# Visualise

In [ ]:
# To show a image
def image(t_image,label,index):
    image_reshaped = t_image.values.reshape(28,28)
    plt.subplot(3,6,index+1)
    plt.imshow(image_reshaped,cmap= plt.cm.gray)
    plt.title(label)
    

plt.figure(figsize=(18,8))
sample = train.sample(18).reset_index(drop=True)

for i,row in sample.iterrows():
    label = row["label"]
    img = row.drop("label")
    image(img,label,i)
    
    
plt.tight_layout()

# Reshaping to numpy array

In [ ]:
X = []
Y = []

for i in train.values:
    X.append(i[1:].reshape(28,28))
    Y.append(i[0])
    
X = np.array(X)
Y = np.array(Y)


In [ ]:

X.shape

In [ ]:
from sklearn.model_selection import train_test_split 

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.2,random_state=42)

# Feature Scale

In [ ]:
X_train = X_train / 255
X_test = X_test / 255

In [ ]:
X_train.shape

In [ ]:
X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1) 

In [ ]:
X_train.shape

In [ ]:
model = keras.Sequential()

model.add(keras.layers.Conv2D(input_shape=(28,28,1),filters=16,kernel_size=(5,5)))
model.add(keras.layers.MaxPool2D((2,2)))
model.add(keras.layers.Conv2D(filters=32,kernel_size=(5,5),padding = 'same',activation='relu'))
model.add(keras.layers.MaxPool2D((2,2)))
model.add(keras.layers.Conv2D(filters=32,kernel_size=(5,5),padding = 'same',activation='relu'))
model.add(keras.layers.MaxPool2D((2,2)))
model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(filters=64 ,kernel_size=(3,3),padding = 'same',activation='relu'))
model.add(keras.layers.MaxPool2D((2,2)))
model.add(keras.layers.Dropout(0.25))
          
model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(300,activation= keras.activations.relu))
model.add(keras.layers.Dropout(0.10))
model.add(keras.layers.Dense(100,activation= keras.activations.relu))
model.add(keras.layers.Dropout(0.15))
model.add(keras.layers.Dense(10,activation= keras.activations.softmax))

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(lr = 0.001),loss=keras.losses.sparse_categorical_crossentropy,metrics=["accuracy"])

history = model.fit(X_train,Y_train,batch_size=250,epochs=10)

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

In [ ]:
prob =model.evaluate(X_test,Y_test)

In [ ]:
 model.save("first.h5")

In [ ]:
from sklearn.metrics import confusion_matrix

t_pred = model.predict(X_test)
t_pred_classes = np.argmax(t_pred,axis=1)
t_pred_classes[:5]
Y_test[:5]  
con_matrix = confusion_matrix(Y_test,t_pred_classes)

In [ ]:
import seaborn as sns
f,ax = plt.subplots(figsize=(8, 8))
sns.heatmap(con_matrix, annot=True, linewidths=0.01,cmap="Greens",linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
pred = model.predict(test_X)
pred_classes = np.argmax(pred,axis=1)


In [ ]:
res = pd.Series(pred_classes,name="Label")
res.shape

In [ ]:
ingId = pd.Series(range(1,28001),name="ImageId")
ingId.shape

In [ ]:
submission = pd.concat([ingId,res],axis=1)


In [ ]:
submission.info()

In [ ]:
submission.head(5)

In [ ]:
submission.to_csv("submission.csv",index=False)